In [1]:
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

df = pd.read_excel("database_cafe_bwi_COMPLETE.xlsx")

# --- SETUP BOT ---
chrome_options = Options()
# chrome_options.add_argument("--headless") 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def get_coords_from_url(url):
    match = re.search(r"@([-?\d\.]+),([-?\d\.]+)", url)
    if match:
        # Langsung gabungin di sini biar praktis
        return f"{match.group(1)}, {match.group(2)}"
    return None

results = []

print("Scraping mulai bekerja...")

for index, row in df.iterrows():
    alamat = row["nama_bisnis"] + ", " + row["alamat"]
    print(f"Mencari: {alamat}")

    try:
        driver.get(f"https://www.google.com/maps/search/{alamat}")
        
        # Tunggu sampai URL berubah ada koordinatnya (@)
        WebDriverWait(driver, 10).until(lambda d: "@" in d.current_url)
        time.sleep(2) 
        
        # Ambil koordinat yang udah digabung
        koordinat = get_coords_from_url(driver.current_url)
        
        results.append(koordinat)
        print(f"Data Koordinat Ditemukan: {koordinat}")
        
    except Exception as e:
        results.append("Gagal/Timeout")
        print(f"Gagal mencari koordinat: {alamat}")

# --- UPDATE DATAFRAME ---
# Karena results itu list biasa, kita tinggal bikin kolom baru di df asal
df["Koordinat"] = results

# --- SIMPAN KE EXCEL ---
df.to_excel("hasil_koordinat_coffee.xlsx", index=False) 

driver.quit()
print("\n Selesai! Cek file 'hasil_koordinat_coffee.xlsx'")

Scraping mulai bekerja...
Mencari: Tomoro Coffee and Pastry Sudirman, Banyuwangi, Jl. Panglima Besar Sudirman No.69, Panderejo, Kec. Banyuwangi, Kabupaten Banyuwangi, Jawa Timur 68432
Data Koordinat Ditemukan: -8.2127877, 114.368863
Mencari: Zazito Co-Working Cafe, Jl. Karang Asem Jl. Tunggul Ametung Gg. I No.02, Taman Baru, Kec. Banyuwangi, Kabupaten Banyuwangi, Jawa Timur 68416
Data Koordinat Ditemukan: -8.2242571, 114.3581515
Mencari: D&A Cafe, Jl. Kepiting No.60, Tukangkayu, Kec. Banyuwangi, Kabupaten Banyuwangi, Jawa Timur 68416
Data Koordinat Ditemukan: -8.229541, 114.3679523
Mencari: Sowan coffee, Kp. Melayu, Kec. Banyuwangi, Kabupaten Banyuwangi, Jawa Timur 68419
Data Koordinat Ditemukan: -8.2066671, 114.378224
Mencari: House Of Coffee Banyuwangi, Jl. Brigjen Katamso No.24, Tukangkayu, Kec. Banyuwangi, Kabupaten Banyuwangi, Jawa Timur 68416
Data Koordinat Ditemukan: -8.2271215, 114.3686691
Mencari: Ndalem Coffee And Roastery Kalasan, Jl. Kalasan No.5, Penganjuran, Kec. Banyuwan